In [174]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchaudio import load
import os

In [175]:
X_train = pd.read_csv('../Files/Standardization/X_Train.csv')
y_train = pd.read_csv('../Files/y_train.csv')
X_test = pd.read_csv('../Files/Standardization/X_Test.csv')
y_test = pd.read_csv('../Files/y_test.csv')

In [176]:
y_train.shape

(856, 2)

In [177]:
# Extract the gender labels from the second column
gender_train = y_train.Gender
gender_test = y_test.Gender

# Check the shape
print(f'Original gender labels shape: {gender_train.shape}, {gender_test.shape}')

# Ensure labels are binary (0 or 1)
# If they are not already binary, use LabelEncoder to convert
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
gender_train = label_encoder.fit_transform(gender_train)
gender_test = label_encoder.transform(gender_test)

# Check the final shapes
print(f'Final gender labels shape: {gender_train.shape}, {gender_test.shape}')


Original gender labels shape: (856,), (129,)
Final gender labels shape: (856,), (129,)


In [178]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the ANN model
ann_model = Sequential([
    Dense(128, input_shape=(39,), activation='relu'),  
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])


ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


ann_history = ann_model.fit(X_train, gender_train, epochs=100, batch_size=32, validation_data=(X_test, gender_test))


Epoch 1/100


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5605 - loss: 0.7213 - val_accuracy: 0.5814 - val_loss: 0.6619
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6298 - loss: 0.6653 - val_accuracy: 0.6124 - val_loss: 0.6447
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6484 - loss: 0.6334 - val_accuracy: 0.6512 - val_loss: 0.6220
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6626 - loss: 0.6216 - val_accuracy: 0.6822 - val_loss: 0.6176
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7291 - loss: 0.5760 - val_accuracy: 0.6977 - val_loss: 0.6029
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7445 - loss: 0.5559 - val_accuracy: 0.7132 - val_loss: 0.5717
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7597 - loss: 0.5258 - val_accuracy: 0.7597 - val_loss: 0.5421
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7913 - loss: 0.4972 - val_accuracy: 0.7597 - val_loss: 0.5

In [179]:
import numpy as np

# Convert DataFrame to numpy array
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

# Reshape for CNN
X_train_cnn = X_train_np.reshape(X_train_np.shape[0], 39, 1, 1) 
X_test_cnn = X_test_np.reshape(X_test_np.shape[0], 39, 1, 1)

print(f'X_train_cnn shape: {X_train_cnn.shape}')  # Should print (856, 39, 1, 1)
print(f'X_test_cnn shape: {X_test_cnn.shape}')    # Should print (129, 39, 1, 1)


X_train_cnn shape: (856, 39, 1, 1)
X_test_cnn shape: (129, 39, 1, 1)


In [180]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


cnn_model = Sequential([
    Conv2D(32, (3, 1), activation='relu', input_shape=(39, 1, 1)),  # (height, width, channels)
    MaxPooling2D((2, 1)),
    Conv2D(64, (3, 1), activation='relu'),
    MaxPooling2D((2, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Single output neuron for binary classification
])


cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


cnn_history = cnn_model.fit(X_train_cnn, gender_train, epochs=100, batch_size=32, validation_data=(X_test_cnn, gender_test))


Epoch 1/100


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5526 - loss: 0.6804 - val_accuracy: 0.6744 - val_loss: 0.6231
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7089 - loss: 0.5976 - val_accuracy: 0.6899 - val_loss: 0.5863
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7450 - loss: 0.5524 - val_accuracy: 0.7132 - val_loss: 0.5460
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7452 - loss: 0.5050 - val_accuracy: 0.7287 - val_loss: 0.5289
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8049 - loss: 0.4608 - val_accuracy: 0.7519 - val_loss: 0.5290
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7969 - loss: 0.4301 - val_accuracy: 0.7519 - val_loss: 0.5075
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8238 - loss: 0.4014 - val_accuracy: 0.7597 - val_loss: 0.5001
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8736 - loss: 0.3523 - val_accuracy: 0.7752 - val_loss: 0.5

In [181]:
ann_loss, ann_accuracy = ann_model.evaluate(X_test, gender_test)
print(f'ANN Model Accuracy: {ann_accuracy * 100:.2f}%')


cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, gender_test)
print(f'CNN Model Accuracy: {cnn_accuracy * 100:.2f}%')


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.8921 - loss: 0.4048
ANN Model Accuracy: 91.47%
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8008 - loss: 1.3406 
CNN Model Accuracy: 84.50%
